# Разрешение семантической неоднозначности

## Сбор данных и предобработка
### Метод получения набора данных
Я взяла корпус примеров из новостей [отсюда](https://nlp4year.wikispaces.com/file/view/news.txt/610208227/news.txt), посчитала частоты встречающихся в нем слов, выбрала два семантически далеких существительных с достаточно большой частотой ("россия" и "компания"), дополнила корпус вручную выбранными примерами из газетного подкорпуса НКРЯ (чтобы совпадал жанр), чтобы на каждое слово приходилось 700 или более примеров, заменила выбранные слова на 'ХХХ'.  
### Характеристика набора данных
+ Жанр - новости.
+ "россия" - 709 контекстов
+ "компания" - 707 контекстов
+ контекст - по 4 токена слева и справа от нужного нам слова
### Предобработка
+ лемматизация с помощью Mystem
+ стоп-слова - слова, принадлежащие к служебным частям речи (предлоги, союзы) и не несущие самостоятельного значения. Список стоп-слов был получен автоматически с помощью данных электронной версии Нового Частотного Словаря Русской Лексики.

In [13]:
from preprocessing import tokenize, sentence_split

In [14]:
from collections import Counter

In [15]:
# собираем контексты
with open('news.txt', 'r') as file:
    corpus_entries = file.readlines()

In [17]:
# препроцессинг: токенизация, лемматизация, удаление стоп-слов
all_tokens = []
for entry in corpus_entries:
    entry = entry.strip()
    entry_tokens = tokenize(entry)
    all_tokens.append(entry_tokens)

In [18]:
cnt  = Counter([element for sublist in all_tokens for element in set(sublist)])

In [19]:
cnt.most_common(15)

[('быть', 1446),
 ('год', 972),
 ('в', 790),
 ('это', 742),
 ('россия', 709),
 ('который', 707),
 ('компания', 707),
 ('рбк', 590),
 ('говорить', 553),
 ('российский', 480),
 ('этот', 469),
 ('по', 410),
 ('может', 406),
 ('банк', 378),
 ('свой', 368)]

In [33]:
# возьмем слова "компания" и "россия", потому что в корпусе достаточно примеров для них
# заменим каждое из слов на XXX
# конекстом будем считать по 4 токена слева и справа от нужного нам слова
# компания - класс 0
# россия - класс 1
entries_with_classes = []
for token_set in all_tokens:
    for i, word in enumerate(['компания', 'россия']):
        if word in token_set:
            token_set_copy = list(token_set)
            ind = token_set_copy.index(word)
            token_set_copy[ind] = 'ХХХ'
            if ind - 4 < 0:
                start = 0
            else:
                start = ind-4
            if ind + 5 > len(token_set_copy):
                finish = len(token_set_copy)
            else:
                finish = ind + 5  
            token_set_copy = token_set_copy[start:finish]
            token_set_copy.remove('ХХХ')
            entries_with_classes.append((' '.join(token_set_copy), i))

In [34]:
entries_with_classes[10]

('можно использовать система банка говорить моисеев она смочь', 1)

In [30]:
len(entries_with_classes)

1416

## Построение векторов контекстов, классификаторы
(CountVectorizer и TF-IDF)   
(Random Forest, Naive Bayes, SVM)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [49]:
import numpy as np

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

In [64]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [41]:
count_vect = CountVectorizer(analyzer='word')
tfidf_vect = TfidfVectorizer(analyzer='word')
vectorizers = [('Count', count_vect), ('TF-IDF', tfidf_vect)]

In [43]:
knn = KNeighborsClassifier()
random_forest = RandomForestClassifier(n_estimators=100)
nb =  MultinomialNB()
classifiers = [('Naive Bayes', nb), ('Random Forest', random_forest), ('KNN', knn)]

In [44]:
X = [entry[0] for entry in entries_with_classes]
y = [entry[1] for entry in entries_with_classes]

In [52]:
def score_classifier(clf, metric='f1'):
    scores = cross_val_score(pipeline, np.asarray(X), np.asarray(y), cv=5, scoring=metric)
    score = sum(scores) / len(scores)
    return score

In [53]:
for clf in classifiers:
    for vctr in vectorizers:
        pipeline = Pipeline([
    ('vectorizer', vctr[1]),
    ('classifier', clf[1])])
        print (clf[0], vctr[0], score_classifier(pipeline))

Naive Bayes Count 0.741429847763
Naive Bayes TF-IDF 0.739871429204
Random Forest Count 0.715719274708
Random Forest TF-IDF 0.724942278036
KNN Count 0.668879128759
KNN TF-IDF 0.68844532505


## Уменьшение размерности векторов
(Возьмем сочетание модели и векторайзера с наилучшим качеством)

In [57]:
# добавим в список стоп-слов полнозначные частотные слова (по Новому Частотному Словарю Русской Лексики 
# в разделе газетно-новостной лексики), которые могут упортребляться
# с одинаковой вероятностью в контектсе с обоими словами (согласно моей языковой интуиции)

stop_words_for_vectorizer = ['президент', 'год', 'известие', 'лидер', 'бюджет', 'рынок', 'союз', 'доллар', 
                            'нынешний', 'автомобиль', 'журналист', 'гонка', 'пресса', 'январь', 'турнир',
                             'рубль', 'декабрь', 'миллион', 'акция', 'состояться', 
                             'строительство', 'сезон', 'премьера', 'тысяча', 'председатель', 'заявить',
                            'корреспондент', 'репетиция', 'октябрь', 'интервью', 'многий', 'итог', 
                             'пресс-конференция', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 
                             'август', 'сентябрь', 'ноябрь', 'месяц', 'неделя', 'планировать', 'планироваться']

In [62]:
# векторайзер с заданными стоп-словами, 
# + не учитывает слова если они встретились более, чем в 25 и менее, чем в 5 документах
vectorizer = CountVectorizer(stop_words = stop_words_for_vectorizer, max_df = 25, min_df = 5)

In [63]:
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', nb)])
print ('Naive Bayes with CountVectorizer and reduced dimension:', score_classifier(pipeline))

Naive Bayes with CountVectorizer and reduced dimension: 0.685491475764


**Качество ухудшилось!**

### Результаты по 7 моделям
+ **Naive Bayes Count 0.741429847763**
+ Naive Bayes TF-IDF 0.739871429204
+ Random Forest TF-IDF 0.724942278036
+ Random Forest Count 0.715719274708
+ KNN TF-IDF 0.68844532505
+ *Naive Bayes Count Reduced: 0.685491475764*
+ KNN Count 0.668879128759

## Ошибки модели
(Посмотрим на ошибки модели с самым высоким качеством предсказания).    
Ошибки возникают в освновном в тех случаях, когда слова, часто встречающиеся в конексте с одним словом появляются в контексте у второго слова или же, когда контекст состоит из слов, которые могут с одинаковой вероятностью встретиться с обоими словами и не несут никакой информации и значении слова. 
Я думаю, что предложенный мной выше способ выбора стоп-слов должен повысить качество классификации, если собрать список стоп-слов более тщательно, использовав большой корпус и автоматически подсчитав, какие слова часто встречаются и со словом "россия", и со словом "компания".   

In [65]:
pipeline = Pipeline([
    ('vectorizer', count_vect),
    ('classifier', nb)])

In [67]:
y_predicted = cross_val_predict(pipeline, X, y)

In [69]:
# компания - класс 0
# россия - класс 1
for i in range(len(y_predicted)):
    if y_predicted[i] != y[i]:
        print('Context: '+X[i], 'True class: '+str(y[i]))

Context: система предмет обсуждение понятно находиться западный юрисдикция цб True class: 0
Context: фактически легализовать статус swift устанавливать он определенный требование True class: 1
Context: мочь заставлять создавать дочерний россия уже быть регулировать True class: 0
Context: заставлять создавать дочерний компания уже быть регулировать этот True class: 1
Context: об это говориться сообщение True class: 0
Context: работа приостанавливать стадия геологоразведка временно отказываться расширение албазино True class: 0
Context: период 2013 год отчитываться прошлый неделя отношение долг True class: 0
Context: останавливаться плановый ремонт наш сегодня принимать чрезвычайный мера True class: 0
Context: минэнерго производство потребление нефтепродукт прозвучать претензия со сторона True class: 1
Context: в пресс-служба рбк сообщать новый срок True class: 0
Context: группа хоум кредит материнский уже раз демонстрировать готовность True class: 0
Context: вход начисляться добавлять с

## Собственные примеры

In [84]:
russia = ['Сборная россии по биатлону заняла 1 место на чемпионате мира.',
         'Россия - священная наша держава',
         'Россия - любимая наша страна',
        'новости Москвы и регионов России',
         'Извини, но ты живёшь в России']
company = ['Группа компаний Дельта предоставляет скидки на оборудование',
          'Adecco - ведущая международная компания, предоставляющая решения в области коммуникаций',
          'Компания и её дочерние предприятия оказывают услуги во всех регионах, в республиках Абхазия, Южная Осетия и Таджикистан.',
          'Обанкротилась компания Vertu, которая специализировалась на производстве телефонов премиум-класса.',
          'Страхование ОСАГО и каско в Москве осуществляют многие страховые компании, предоставляющие услуги автострахования.']

In [85]:
both = russia+company

In [86]:
both_preprocessed = []
for entry in both:
    token_set = tokenize(entry)
    for i, word in enumerate(['компания', 'россия']):
        if word in token_set:
            token_set_copy = list(token_set)
            ind = token_set_copy.index(word)
            token_set_copy[ind] = 'ХХХ'
            if ind - 4 < 0:
                start = 0
            else:
                start = ind-4
            if ind + 5 > len(token_set_copy):
                finish = len(token_set_copy)
            else:
                finish = ind + 5  
            token_set_copy = token_set_copy[start:finish]
            token_set_copy.remove('ХХХ')
            both_preprocessed.append(' '.join(token_set_copy))

In [87]:
both_preprocessed

['сборная биатлон занимать 1 место',
 'священный наш держава',
 'любимый наш страна',
 'новость москва регион',
 'извинять жить',
 'группа дельта предоставлять скидка оборудование',
 'adecco ведущая международный предоставлять решение область коммуникация',
 'она дочерний предприятие оказывать',
 'обанкротиться vertu который специализироваться производство',
 'каско москва осуществлять страховой предоставлять услуга автострахование']

In [88]:
pipeline.fit(X, y)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [89]:
y_my_examples = pipeline.predict(both_preprocessed)

In [90]:
y_my_examples_true = [1,1,1,1,1,0,0,0,0,0]

In [91]:
len(y_my_examples)

10

In [94]:
from sklearn.metrics import classification_report, confusion_matrix

In [93]:
print(classification_report(y_my_examples_true, y_my_examples))

             precision    recall  f1-score   support

          0       0.71      1.00      0.83         5
          1       1.00      0.60      0.75         5

avg / total       0.86      0.80      0.79        10



In [97]:
print(confusion_matrix(y_my_examples_true, y_my_examples, labels=[0, 1]))

[[5 0]
 [2 3]]


In [98]:
tn, fp, fn, tp = confusion_matrix(y_my_examples_true, y_my_examples, labels=[0, 1]).ravel()

In [99]:
(tn, fp, fn, tp)

(5, 0, 2, 3)